## 此檔案適用於1111網站

由於1111網站中所存有的公司較少，因此取完資料後並不會再次比對公司名單，
流程即
1. 取得公司介紹頁面網址
2. 解碼電話號碼
3. 讀取文字內容
於此處的傳真部分，因為無法得知編碼方式。因此無法取得。
於此網站中的資訊依各公司而有所不同，部分公司因頁面設定原因無法得到有用資訊，詳細資料可參考執行過程中的輸出內容。


#### read the name of all the company from a .txt file

In [ ]:
with open('company_name_list.txt', 'r') as f:
    lines = f.readlines()
    for i in range(len(lines)):
        print(lines[i].strip())

#### import the required packages

In [1]:
import time
import datetime
import requests
import re
import json
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import os
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import urllib.parse
from urllib.parse import quote, unquote, urljoin, urlparse, parse_qs, urlencode, urlunparse
import csv
import base64

### 主要功能部分

步驟說明:
1. 將名單內之公司名稱透過url encode 傳入url並查詢
2. 取得公司對應之代號並加入前段固定內容
3. 解碼公司電話(此處使用base64編碼)
4. 取得各欄位對應之值
5. 存入csv檔中


In [2]:
##1. 將名單內之公司名稱透過url encode 傳入url並查詢
##2. 取得公司對應之代號並加入前段固定內容
def get_info_url(keyword):
    # URL encode the keyword for the search query
    encoded_keyword = quote(keyword)
    # Construct the search URL
    url = f'https://www.1111.com.tw/search/corp?ks={encoded_keyword}'
    res = requests.get(url)
    
    if res.status_code == 200:
        print(res.status_code)  # Print status code for debugging
        
        soup = BeautifulSoup(res.text, "html.parser")
        parent_div = soup.find('div', class_='title')
        
        if parent_div:
            a_tag = parent_div.find('a')
            
            if a_tag:
                link_text = a_tag.get_text(strip=True)
                print(f"Link text: {link_text}")  # Print link text for debugging
                
                # Check if the link text contains the keyword
                if keyword in link_text:
                    relative_href = a_tag.get('href')
                    print(f"Relative URL: {relative_href}")  # Print the relative URL for debugging
                    
                    # Construct the full URL
                    full_url = urljoin('https://www.1111.com.tw', relative_href)
                    return full_url
                else:
                    print(f"Keyword '{keyword}' not found in the link text.")
    else:
        print(f"Failed to retrieve the URL, status code: {res.status_code}")
    
    return None


In [3]:
##function to decode the phone number
##3. 解碼公司電話(此處使用base64編碼)
def decode_base64(encoded_str):
    """Decode a Base64 encoded string."""
    decoded_bytes = base64.b64decode(encoded_str)
    decoded_str = decoded_bytes.decode('utf-8')
    return decoded_str

In [4]:
## get the content
##取得各欄位對應之值
def get_all_children(公司名稱, soup,num_work,div_class='app'):
    # response = requests.get(base_url)
    # if response.status_code == 200:
    #     soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the parent div with the specified class
        parent_div = soup.find('div', id=div_class)
        if parent_div:
            # Find the child article with the specified class
            child_article = parent_div.find('article', class_='information padding_60')
            if child_article:
                # Find all <li> elements within the child article
                ul_items = child_article.find_all('li')
                
                data = {
                    '公司名稱': 公司名稱,
                    '產業類別': '',
                    '產業說明': '',
                    '電話': '',
                    '傳真': '',
                    '資本額': '',
                    '員工人數': '',
                    '公司地址': '',
                    '聯絡人': '',
                    '工作機會':num_work
                }
                
                for li in ul_items:
                    key_span = li.find('span')
                    value_span = li.find_all('span')[1] if len(li.find_all('span')) > 1 else None
                    value_div = li.find('div', class_='d-flex flex-wrap')  # Look for div containing location or category links
                    key = key_span.get_text(strip=True) if key_span else None

                    if key:
                        if value_span:
                            value = value_span.get_text(strip=True)

                            if key == '電話' or key == '傳真':
                                # Decode encoded phone or fax numbers
                                encoded_value = value_span.get('data-tel')
                                if encoded_value:
                                    decoded_value = decode_base64(encoded_value)
                                    data[key] = decoded_value
                            else:
                                # For other fields, just use the text
                                if key in data:
                                    data[key] = value
                        elif value_div:
                            if key == '公司位置':
                                # Extract and concatenate multiple <a> tags
                                location_texts = [a.get_text(strip=True) for a in value_div.find_all('a')]
                                data['公司地址'] = ' '.join(location_texts)
                            elif key == '產業類別':
                                # Extract and concatenate text from <a> tags within the <div>
                                category_texts = [a.get_text(strip=True) for a in value_div.find_all('a')]
                                data['產業類別'] = ' '.join(category_texts)
                        elif key == '產業類別':
                            # Extract and concatenate text from <a> tags directly under <li>
                            a_tags = li.find_all('a')
                            if a_tags:
                                category_texts = [a.get_text(strip=True) for a in a_tags]
                                data['產業類別'] = ' '.join(category_texts)
                            
                return data
    # return None


In [5]:
def get_work_opportunity(soup, id='Jobs'):
    try:
        # response = requests.get(url)
        # response.raise_for_status()  # Raise an exception for HTTP errors

        # soup = BeautifulSoup(response.text, 'html.parser')

        parent_div = soup.find('div', id=id)
        if parent_div:
            span_tag = parent_div.find('h2').find('span')
            if span_tag:
                # Extract and return the text from the <span> tag
                text = span_tag.get_text(strip=True)
                return text
            else:
                print("Can't find the <span> tag within the <h2> tag")
                return '0'
        else:
            print(f"Can't find the div with the id: {id}")
            return '0'

    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return None

# # Example usage
# base_url = 'https://www.1111.com.tw/corp/71716508/'
# result = get_work_opportunity(base_url)
# print(result)


In [6]:

##5. 存入csv檔中
def save_to_csv(data, file_name):
    """Save data to a CSV file. Write header only if the file is being created."""
    # Check if the file already exists
    file_exists = os.path.isfile(file_name)
    
    with open(file_name, mode='a', newline='', encoding='utf-8') as file:
        file.write('\ufeff') ##確保透過excel開啟時編碼正確
        writer = csv.DictWriter(file, fieldnames=data.keys())
        
        # Write header only if the file does not already exist
        if not file_exists:
            writer.writeheader()
        
        # Write the data row
        writer.writerow(data)

In [7]:
if __name__ == "__main__":
  # csv_file_name = 'result_1111_new.csv'
  full_url = ''
  公司名稱 = ''
  產業類別 = ''
  產業描述  = ''
  公司電話 = ''
  資本額 = ''
  員工人數 =''
  公司網址 = ''
  公司地址 =''
  聯絡人 = ''
  工作機會 =''
  with open('company_new.txt', 'r') as f:
    lines = f.readlines()
    for i in range(len(lines)):
    # for i in range(3):
        公司名稱 = lines[i].strip()
        print(f'公司名稱 ：{公司名稱}')
        keyword = lines[i].strip()
        first_url = get_info_url(keyword)
        print(first_url)
        if first_url != None:
          response = requests.get(first_url)
          if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            工作機會 = get_work_opportunity(soup)
            print(f"工作機會:{工作機會}")
            match = re.search(r'\d+[\+\d]*', 工作機會)
            if match:
                num_work = match.group()
                print(f"num:{num_work}")
            else:
                num_work = 0
                print("Number not found in the text")
            data = get_all_children(公司名稱,soup,num_work)
            if data:
              print("Extracted Data:")
              print(data)
      
            # Save to CSV
              csv_file_name = 'result_1111_new_ttt.csv'
              save_to_csv(data, csv_file_name)
            else:
              print("Failed to retrieve or process data.")


公司名稱 ：輝寶實業有限公司
200
None
公司名稱 ：健鼎科技股份有限公司
200
Link text: 健鼎科技股份有限公司
Relative URL: /corp/2132334/
https://www.1111.com.tw/corp/2132334/
工作機會:工作機會 ( 共 34 個職缺 )
num:34
Extracted Data:
{'公司名稱': '健鼎科技股份有限公司', '產業類別': '印刷電路板製造業(PCB)', '產業說明': '', '電話': '(03)4195678', '傳真': '', '資本額': '52億', '員工人數': '25000人', '公司地址': '[平鎮工業區] 桃園市平鎮區工業五路21號', '聯絡人': '', '工作機會': '34'}
公司名稱 ：建樺群業股份有限公司
200
None
公司名稱 ：安琪兒百貨有限公司
200
None
公司名稱 ：明揚特殊氣體有限公司
200
None
公司名稱 ：全聯旅行社股份有限公司
200
None
公司名稱 ：聖俯建設股份有限公司
200
None
公司名稱 ：台灣晶亮隱形眼鏡股份有限公司
200
None
公司名稱 ：新新憶企業股份有限公司
200
None
公司名稱 ：通盈通運股份有限公司
200
Link text: 通盈通運股份有限公司(總公司)
Relative URL: /corp/8930009/
https://www.1111.com.tw/corp/8930009/
工作機會:工作機會 ( 共 34 個職缺 )
num:34
Extracted Data:
{'公司名稱': '通盈通運股份有限公司', '產業類別': '儲配╱運輸物流', '產業說明': '', '電話': '', '傳真': '', '資本額': '5000萬', '員工人數': '400人', '公司地址': '新北市五股區新五路二段130號', '聯絡人': '', '工作機會': '34'}
公司名稱 ：稻田營造有限公司
200
Link text: 稻田營造有限公司
Relative URL: /corp/50018191/
https://www.1111.com.tw/corp/50018191/
工作機會:工作機會 ( 共 18 個職缺 )
nu